# Simple tests for the planar diagnostics

Note - sometimes if you're visually comparing the output files with numbers calculated in numpy beware that apparent small differences can just be due to  the precision in the writeout. If you like change the line `const int outfilePrecision = 10;` in `MaestroDiag.cpp` to have more precision.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import yt
##yt.mylog.setLevel()
yt.utilities.logger.disable_stream_logging()

In [2]:
dataDir = 'single_level/'
baseName = 'plt'

In [3]:
ds = yt.load(dataDir + baseName + '0000001') # LOAD AT PLT 1 - intended that that corresponds to step==1

In [4]:
#ds.field_list

In [5]:
dx, dy, dz = ds.domain_width / ds.domain_dimensions # think this is ok for single level / UG

## diag_vel.out

Look at the file (assuming its only a few rows)

In [6]:
cat diag_vel.out

time                max{U}              max{Mach}           tot kin energy      tot grav energy     tot int energy      dt                  
0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    2.6425129002e+21    9.4816020347e+30    1.9887378221e-01    
1.9887378221e-01    5.5741349954e-24    7.8830693304e-31    2.0558712522e-30    2.6425129002e+21    9.4816020347e+30    1.9887378221e-01    
4.1763494264e-01    6.1172091860e-24    8.6510972844e-31    2.4786167803e-30    2.6425129002e+21    9.4816020347e+30    2.1876116043e-01    
6.5827221911e-01    6.7287893357e-24    9.5160079343e-31    2.9970504039e-30    2.6425129002e+21    9.4816020347e+30    2.4063727647e-01    
9.2297322323e-01    7.4061684739e-24    1.0473973020e-30    3.6311173749e-30    2.6425129002e+21    9.4816020347e+30    2.6470100412e-01    
1.2141443278e+00    8.1461824805e-24    1.1520517771e-30    4.3956320691e-30    2.6425129002e+21    9.4816020347e+30    2.9117110453e-01    
1.5344325427e

Load up data needed for velocities and kinetic energy calculation

In [7]:
all_data_level_0 = ds.covering_grid(level=0, left_edge=[0,0.0,0.0], dims=[64, 64, 1])
vx = all_data_level_0['velx']
vx = vx[:,:,0].transpose()
vy = all_data_level_0['vely']
vy = vy[:,:,0].transpose()
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
w0y = all_data_level_0['w0y']
w0y = w0y[:,:,0].transpose()
vel = np.sqrt(vx**2 + (vy+w0y)**2)

Whats the maximum velocity according to the plotfile data? (Compare this to the max{U} col at the correspoding time)

In [8]:
np.max(vel) 

5.574134995444917e-24 dimensionless

Whats the total kinetic energy according to the data in the plotfile? (The recorded value by the code currently ignores the factor 0.5, and will Normalise for dA or dV)
Compare this to the "tot kin energy" column at the appropriate time)

In [9]:
ke = 0.5 * rho * vel**2
np.sum(2. * ke *dx * dy)

2.0558712521914724e-30 code_length**2

Whats the total internal energy according to the data in the plotfile? I had to look inside the EOS code to see how this is calculated

In [10]:
T = all_data_level_0['tfromp']
T = T[:,:,0].transpose()

In [11]:
k_B = 1.3806488e-16   # erg/K
n_A = 6.02214129e23  # mol^-1
R = k_B * n_A

mu = 1

gamma = 5./3.
cv = R /mu / (gamma-1)
eint = cv * T 

In [12]:
test = np.sum(eint*rho * dx**2)

In [13]:
np.float(test)


9.481602034658219e+30

Whats the maximum mach number?

In [14]:
rho = all_data_level_0['rho']
rho = rho[:,:,0].transpose()
p = all_data_level_0['p0']
p = p[:,:,0].transpose()
cs = np.sqrt(gamma * p / rho)
mach = vel / cs

print( np.max(mach))

7.8830693304476635e-31 dimensionless


### Whats the gravitational potential energy?

#### Definition

Note $g=-\frac{d \Phi(r)}{dr}$ with boundary condition $\Phi(r)=0$ (wlog) and constant gravity gives $\Phi=-gr$.

Note that $\Phi$ has units of length squared / time squared (in cgs cm^2/s^2) which can be thought of as energy per unit mass

So an energy density can be defined 

$\rho \Phi = - \rho g r$ 

This then can be evaluated with an appropriate volume integral (or area integral for a "2D energy")

#### Anayltical expectation for the lgw base state

The integral along the base state line is

$GE = -g \int_0^H \rho_0 r dr = -g \rho_{base} \int_0^H r e^{-r/H} dr $

$= -g \rho_{base} \left[-He^{-r/H}(H+R)\right]_0^{H}$

$= -g \rho_{base} H^2 [1-2e^{-1}] = -g \rho_{base} H^2 \times 0.26424111765$

This should then probabally be multiplied by the length in x to account for the rest of the integral over our 2d plane



For our implementation in maestro it seems pretty close, and I expect the difference is more to due with the numeric representation of our nominal base state rather than the way in which we calculate grav energy:

In [15]:
g=-3e4 # if the input file is changed these need to be
H=1e9
r=1/3

U = -g * r * H**2 * (1-2*np.exp(-1))
print(U, U*1e9)

2.6424111765711534e+21 2.642411176571153e+30


In [16]:
Uout = 2.6425129002e21 #version manually entered copy and pasted from diag file
print(Uout / U)
print((Uout-U)/U)
print((Uout-U)*1e9)

1.000038496517782
3.84965177820839e-05
1.0172362884644864e+26


## diag_temp.out

In [17]:
cat diag_temp.out

time                max{T}              x(max{T})           y(max{T})           z(max{T})           vx(max{T})          vy(max{T})          vz(max{T})          
0.0000000000e+00    3.6081708564e+05    7.8125000000e+06    7.8125000000e+06    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    0.0000000000e+00    
1.9887378221e-01    3.6081708564e+05    7.8125000000e+06    7.8125000000e+06    0.0000000000e+00    -5.3180202686e-25   -1.5263408823e-24   0.0000000000e+00    
4.1763494264e-01    3.6081708564e+05    7.8125000000e+06    7.8125000000e+06    0.0000000000e+00    -6.0777374498e-25   -1.6472049793e-24   0.0000000000e+00    
6.5827221911e-01    3.6081708564e+05    7.8125000000e+06    7.8125000000e+06    0.0000000000e+00    -6.6302590361e-25   -1.7956951556e-24   0.0000000000e+00    
9.2297322323e-01    3.6081708564e+05    7.8125000000e+06    7.8125000000e+06    0.0000000000e+00    -7.2518458208e-25   -1.9614516315e-24   0.0000000000e+00    
1.2141443278e+00    3.6081708564e+

In [18]:
T = all_data_level_0['tfromh']
T = T[:,:,0].transpose()
print(np.max(T))

360817.0856409371 dimensionless


can have a look at the actual coordinates, but its tricky if there are multiple maxima - might want to setup a test where one cell is very hot?

In [19]:
iy,ix = np.where( T == np.max(T))

In [20]:
T[iy, ix]

YTArray([360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.08564094, 360817.08564094,
         360817.08564094, 360817.0856409

In [21]:
ix * dx + dx/2

YTArray([7.812500e+06, 2.343750e+07, 3.906250e+07, 5.468750e+07,
         7.031250e+07, 8.593750e+07, 1.015625e+08, 1.171875e+08,
         1.328125e+08, 1.484375e+08, 1.640625e+08, 1.796875e+08,
         1.953125e+08, 2.109375e+08, 2.265625e+08, 2.421875e+08,
         2.578125e+08, 2.734375e+08, 2.890625e+08, 3.046875e+08,
         3.203125e+08, 3.359375e+08, 3.515625e+08, 3.671875e+08,
         3.828125e+08, 3.984375e+08, 4.140625e+08, 4.296875e+08,
         4.453125e+08, 4.609375e+08, 4.765625e+08, 4.921875e+08,
         5.078125e+08, 5.234375e+08, 5.390625e+08, 5.546875e+08,
         5.703125e+08, 5.859375e+08, 6.015625e+08, 6.171875e+08,
         6.328125e+08, 6.484375e+08, 6.640625e+08, 6.796875e+08,
         6.953125e+08, 7.109375e+08, 7.265625e+08, 7.421875e+08,
         7.578125e+08, 7.734375e+08, 7.890625e+08, 8.046875e+08,
         8.203125e+08, 8.359375e+08, 8.515625e+08, 8.671875e+08,
         8.828125e+08, 8.984375e+08, 9.140625e+08, 9.296875e+08,
         9.453125e+08, 9.

In [22]:
iy * dy + dy/2

YTArray([7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500., 7812500., 7812500.,
         7812500., 7812500., 7812500., 7812500.]) code_length

In [23]:
vx[iy,ix]

YTArray([-5.31802027e-25, -4.04031410e-25, -4.17844450e-25,
         -4.62736829e-25, -4.48923789e-25, -4.21297710e-25,
         -4.00578150e-25, -3.66045551e-25, -3.55685771e-25,
         -3.28059692e-25, -3.10793392e-25, -2.72807533e-25,
         -2.79714053e-25, -2.41728194e-25, -2.48634714e-25,
         -2.07195595e-25, -2.03742335e-25, -1.69209736e-25,
         -1.65756476e-25, -1.38130397e-25, -1.41583657e-25,
         -1.17410837e-25, -1.17410837e-25, -1.07051057e-25,
         -1.07051057e-25, -9.32380177e-26, -1.03597797e-25,
         -8.63314979e-26, -8.28782380e-26, -6.90651983e-26,
         -9.32380177e-26, -7.25184582e-26, -6.90651983e-26,
         -5.17988987e-26, -5.17988987e-26, -3.79858591e-26,
         -5.17988987e-26, -1.03597797e-26, -4.14391190e-26,
         -1.72662996e-26, -3.45325991e-27, -3.45325991e-27,
         -6.90651983e-27,  4.48923789e-26,  3.10793392e-26,
          9.32380177e-26,  9.32380177e-26,  1.45036916e-25,
          1.62303216e-25,  2.52087974e-2

In [24]:
vy[iy, ix] + w0y[iy,ix]

YTArray([-1.52634088e-24, -1.58159304e-24, -1.60921912e-24,
         -1.59540608e-24, -1.56778000e-24, -1.55396696e-24,
         -1.53324740e-24, -1.51943436e-24, -1.51598110e-24,
         -1.50216806e-24, -1.49526154e-24, -1.48835502e-24,
         -1.47799524e-24, -1.47108872e-24, -1.46763546e-24,
         -1.47108872e-24, -1.46418220e-24, -1.46418220e-24,
         -1.46072894e-24, -1.45382242e-24, -1.45382242e-24,
         -1.45382242e-24, -1.44346264e-24, -1.43310286e-24,
         -1.43655612e-24, -1.43655612e-24, -1.43310286e-24,
         -1.43655612e-24, -1.44000938e-24, -1.44691590e-24,
         -1.44691590e-24, -1.45036916e-24, -1.44346264e-24,
         -1.42964960e-24, -1.43310286e-24, -1.44000938e-24,
         -1.42964960e-24, -1.44346264e-24, -1.44000938e-24,
         -1.44346264e-24, -1.45036916e-24, -1.45036916e-24,
         -1.44346264e-24, -1.44691590e-24, -1.44000938e-24,
         -1.44691590e-24, -1.45382242e-24, -1.45727568e-24,
         -1.46418220e-24, -1.47799524e-2

## diag_enuc.out

Dont really have a good test for this (we have no burning). The code is logically to temp, though, so the rough idea is probabally fine but I'd check yourself if you start using diag_enuc for a planar setup.